### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [3]:
counts = reviews.groupby('movie_id')['rating'].count()
reviews['count'] = [counts[x] for x in reviews['movie_id'] ]


In [4]:
reviews['rating'][reviews['movie_id'] == 417]#.columns
temp = reviews.groupby('movie_id')['rating'].count()
reviews['date'] = pd.to_datetime(reviews['date'])


In [5]:
reviews['mean_rate'] = reviews.groupby('movie_id')['rating'].mean()

result_by_rating_and_count = reviews.sort_values(by=['mean_rate','count','date'], ascending = False)
#print(result_by_rating_and_count.head())

result_by_rating_and_count_5 = result_by_rating_and_count[result_by_rating_and_count['count'] >= 5]
print(result_by_rating_and_count_5.head())

#t = movies[movies['movie_id'].isin(result_by_rating_and_count_5.index)].sort_values(by = 'date', ascending=False)
#t["date"][t['date'][t['date']==2018].duplicated().values == True]


        user_id  movie_id  rating   timestamp                date  month_1  \
20581      1907   1454468      10  1381504328 2013-10-11 15:12:08        0   
90770      6905    993846       9  1419909611 2014-12-30 03:20:11        0   
50818      4063    770828      10  1514407576 2017-12-27 20:46:16        0   
110251     8568    770828       7  1458938016 2016-03-25 20:33:36        0   
17668      1652    770828       9  1383536744 2013-11-04 03:45:44        0   

        month_2  month_3  month_4  month_5    ...      month_11  month_12  \
20581         0        0        0        0    ...             0         0   
90770         0        0        0        0    ...             0         1   
50818         0        0        0        0    ...             0         1   
110251        0        0        0        0    ...             0         0   
17668         0        0        0        0    ...             1         0   

        year_2013  year_2014  year_2015  year_2016  year_2017  year_

In [59]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    #append a count column for the number of ratings
    counts = reviews.groupby('movie_id')['rating'].count()

    #get movies by the highest average rating then by the highest count and by date, greater than 5 reviews 
    means = reviews.groupby('movie_id')['rating'].mean()
    last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
    last_rating.columns = ['last_rate_date']
    
    mean_counts_df = pd.DataFrame({'counts':counts,'mean_rating':means})
    mean_counts_date_df = mean_counts_df.join(last_rating)
    #join everything to movies for sorting
    final_df = movies.set_index('movie_id').join(mean_counts_date_df)
    
    result_by_rating_count_date = final_df.sort_values(by=['mean_rating','counts','last_rate_date'], ascending = False)
    
    result_greater_than5 = result_by_rating_count_date[result_by_rating_count_date['counts'] >= 5]
        
    #if rating any count are similar, sort by movie date
    top_movies = result_greater_than5[:n_top]
    
    return list(top_movies['movie']) # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [60]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations('1', 20) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations('53968', 5)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('70000', 100)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35)# Your solution list here



In [61]:
recs_20_for_1

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

In [62]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

['MSG 2 the Messenger (2015)', 'Avengers: Age of Ultron Parody (2015)', 'Sorry to Bother You (2018)', 'Selam (2013)', "Quiet Riot: Well Now You're Here, There's No Way Back (2014)", 'Crawl Bitch Crawl (2012)', 'Make Like a Dog (2015)', 'Pandorica (2016)', 'Third Contact (2011)', 'Romeo Juliet (2009)', 'Be Somebody (2016)', 'Birlesen Gonuller (2014)', 'Agnelli (2017)', 'Sátántangó (1994)', 'Shijie (2004)', 'Foster (2011)', 'CM101MMXI Fundamentals (2013)', 'Akahige (1965)', 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)', 'Körkarlen (1921)']
If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [64]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    REDO THIS DOC STRING
    
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # create top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies


In [65]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

# Top 5 movies recommended for id 53968 with no genre filter but years=['2015', '2016', '2017', '2018']
recs_5_for_53968_filtered = popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018'])

# Top 100 movies recommended for id 70000 with no year filter but genres=['History', 'News']
recs_100_for_70000_filtered = popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News'])



In [66]:
### You Should Not Need To Modify Anything In This Cell

# check 1 
assert t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History']) == recs_20_for_1_filtered,  "The first check failed..."
# check 2
assert t.popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018']) == recs_5_for_53968_filtered,  "The second check failed..."
# check 3
assert t.popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News']) == recs_100_for_70000_filtered,  "The third check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


In [68]:
ranked_movies

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.000000,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,28,2016-01-08 00:44:43
5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.000000,14,2018-06-17 01:44:48
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,10,2015-05-10 22:56:01
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,10.000000,6,2016-01-23 00:30:44
2219210,Crawl Bitch Crawl (2012),Horror|Sci-Fi|Thriller,2012,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,10.000000,6,2013-07-22 23:30:52
4448444,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,5,2017-09-09 13:51:48
5131914,Pandorica (2016),Sci-Fi,2016,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,5,2016-04-04 05:52:22
2059318,Third Contact (2011),Mystery|Sci-Fi|Thriller,2011,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,10.000000,5,2014-02-13 15:48:31
